# 感情分析に基づく分類

## パッケージのインストール

In [1]:
!pip install oseti

You should consider upgrading via the '/Users/tksakaki/.pyenv/versions/3.9.10/envs/ginza4.0/bin/python3.9 -m pip install --upgrade pip' command.


## 基本的な使い方

In [2]:
import oseti

analyzer = oseti.Analyzer()
print(analyzer.analyze_detail('天国で待ってる。'))
print(analyzer.analyze_detail('遅刻したけど楽しかったし嬉しかった。すごく充実した！'))

[{'positive': ['天国'], 'negative': [], 'score': 1.0}]
[{'positive': ['楽しい', '嬉しい'], 'negative': ['遅刻'], 'score': 0.3333333333333333}, {'positive': ['充実'], 'negative': [], 'score': 1.0}]


In [3]:
from collections import Counter
import oseti

filename = 'text/kageotoko.corpus.txt'
analyzer = oseti.Analyzer()

with open(filename, 'r', encoding='UTF-8') as f:
    hist = Counter(
        element['score']
        for line in f
        for element in analyzer.analyze_detail(line))

print('score count')
print('-----------')
for k in sorted(hist.keys()):
    print('{:>5.02f} {}'.format(k, hist[k]))

score count
-----------
-1.00 1047
-0.67 1
-0.60 7
-0.56 1
-0.50 12
-0.43 1
-0.33 67
-0.20 6
-0.14 3
 0.00 3164
 0.20 7
 0.33 78
 0.43 1
 0.50 12
 0.60 6
 1.00 989


# トピックモデルによるクラスタリング

## パッケージのインストール

In [4]:
!pip install gensim pandas openpyxl
!pip install ginza ja-ginza

You should consider upgrading via the '/Users/tksakaki/.pyenv/versions/3.9.10/envs/ginza4.0/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/tksakaki/.pyenv/versions/3.9.10/envs/ginza4.0/bin/python3.9 -m pip install --upgrade pip' command.


## 解析対象データをダウンロード

In [5]:
!mkdir -p sisyou_db

for year in ['h18', 'h19', 'h20', 'h21', 'h22', 'h23', 'h24']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        !wget -P sisyou_db -c https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_{year}_{month}.xls

for year in ['h25', 'h26', 'h27', 'h28', 'h29']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        !wget -P sisyou_db -c https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_{year}_{month}.xlsx

## サンプルコードでは平成29年1月分のデータしか利用しませんが、
## データとしては平成18年〜平成29年の12年分のデータをダウンロードします。
## ぜひご自身でも分析してみてください、

--2022-03-09 18:06:37--  https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_h18_01.xls
anzeninfo.mhlw.go.jp (anzeninfo.mhlw.go.jp) をDNSに問いあわせています... 133.162.110.156
anzeninfo.mhlw.go.jp (anzeninfo.mhlw.go.jp)|133.162.110.156|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 1452544 (1.4M) [application/vnd.ms-excel]
`sisyou_db/sisyou_db_h18_01.xls' に保存中

sisyou_db_h18_01.xl 100%[===================>]   1.38M  --.-KB/s 時間 0.09s      

2022-03-09 18:06:37 (15.5 MB/s) - `sisyou_db/sisyou_db_h18_01.xls' へ保存完了 [1452544/1452544]

--2022-03-09 18:06:38--  https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_h18_02.xls
anzeninfo.mhlw.go.jp (anzeninfo.mhlw.go.jp) をDNSに問いあわせています... 133.162.110.156
anzeninfo.mhlw.go.jp (anzeninfo.mhlw.go.jp)|133.162.110.156|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 1411072 (1.3M) [application/vnd.ms-excel]
`sisyou_db/sisyou_db_h18_02.xls' に保存中

sisyou_db_h18_02.xl 100%[===================>]   1.35M  --.-KB/s 時間

## 基本的な使い方

In [6]:
import pandas
df=pandas.read_excel('sisyou_db/sisyou_db_h29_01.xlsx')
print(df['災害状況'])

0                                                     NaN
1       工場内で床に置いていたコードに、荷物を抱えていた状態のときに足が引っ掛かり、よろめいて数歩前...
2       倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って...
3       会社構内にて車輌の洗車中、足を滑らせ転倒した際に左手をつき、翌朝に左肩の痛みが大きくなり、左...
4                       厩舎2階でバッカン受け入れ作業中、バッカンを落とす穴から落下した。
                              ...                        
2691    重機の整備中、待機している台船へ乗船時に、つまずいて高さ1m40㎝～50㎝の所から転落し、足...
2692    新聞配達中、アパートにて2階と3階の配達を終え、1階に下りる時に誤って最後の段で足を滑らせて...
2693    左手にしびれを感じ、中指にも痛みが出始めたたため検査した結果、手根管症候群と中指ばね指と診断...
2694    塗装場所へ移動する為、5尺の脚立をはしご状態にして、約2.3m程上がった屋上へ上る途中に使用...
2695    入浴介助後、利用者（男性48㎏・全介助・車いす）を洗い場から車いすに移動させる際、新人職員が...
Name: 災害状況, Length: 2696, dtype: object


In [7]:
import logging
import spacy
import pandas as pd 

# 1. 学習中の状況を出力するフォーマットの指定
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

# 2. 解析器の初期化
nlp = spacy.load("ja_ginza")

# 3. ドキュメントを読み込んで単語分割する
df=pd.read_excel('sisyou_db/sisyou_db_h29_01.xlsx')
docs=[]
for text in df['災害状況']:
    if type(text) != str: continue
    doc=[token.lemma_ for token in nlp(text)]
    docs.append(doc)

#
# 4. 極端に頻度が低い語や高い語を除外する
#
from gensim.corpora import Dictionary

# 4a. 辞書の作成
dictionary = Dictionary(docs)

# 4b. 出現が20文書に満たない単語と50%以上の文書に出現する単語を極端とみなして除外する
dictionary.filter_extremes(no_below=20, no_above=0.5)

# 4c. 上記のように定められた語彙で、文書単語行列(Bag-of-words表現)を求める
corpus = [dictionary.doc2bow(doc) for doc in docs]

print(f'Number of unique tokens: {len(dictionary)}')
print(f'Number of documents: {len(corpus)}')

#
# 5. バイグラムの計算
#
from gensim.models import Phrases

# 5a. 文書単語行列にバイグラムを加える (出現頻度20以上のもの)
phrase_model = Phrases(docs, min_count=20)
for idx, doc in enumerate(docs):
    # _が含まれるトークンはバイグラム(あるいはそれ以上)なので、追加する
    bigram_tokens = [token for token in phrase_model[doc] if '_' in token]
    docs[idx].extend(bigram_tokens)

#            
# 6. LDAモデル
#
from gensim.models import LdaModel

num_topics = 10

temp = dictionary[0]  # 辞書をメモリに読み込むための処理

# 6a. LDAモデルの計算
model = LdaModel(
    corpus=corpus,
    id2word=dictionary.id2token,
    chunksize=2000,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=num_topics,
    passes=20,
)

# 6b. LDAモデルによって得られるトピックの抽出
top_topics = model.top_topics(corpus) 
coherences = [coherence for topic_words, coherence in top_topics]

# 6c. 評価指標コヒーレンスを求める
avg_topic_coherence = sum(coherences) / num_topics
print(f'Average topic coherence: {avg_topic_coherence:.4f}.')

2022-03-09 18:08:20,070 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-09 18:08:20,213 : INFO : built Dictionary(7284 unique tokens: ['、', '。', 'ある', 'いる', 'が']...) from 2695 documents (total 149091 corpus positions)
2022-03-09 18:08:20,213 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(7284 unique tokens: ['、', '。', 'ある', 'いる', 'が']...) from 2695 documents (total 149091 corpus positions)", 'datetime': '2022-03-09T18:08:20.213870', 'gensim': '4.1.2', 'python': '3.9.10 (main, Jan 24 2022, 11:22:25) \n[Clang 12.0.0 (clang-1200.0.32.29)]', 'platform': 'macOS-10.15.7-x86_64-i386-64bit', 'event': 'created'}
2022-03-09 18:08:20,220 : INFO : discarding 6666 tokens: [('、', 2673), ('。', 2689), ('いる', 1436), ('が', 1856), ('する', 2506), ('た', 2661), ('て', 2264), ('で', 1680), ('に', 2504), ('の', 2328)]...
2022-03-09 18:08:20,221 : INFO : keeping 618 tokens which were in no less than 20 and no more than 1347 (=50.0%) documents
2022-03-09 18:08:20,224 : INFO : re

Number of unique tokens: 618
Number of documents: 2695


2022-03-09 18:08:20,767 : INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
2022-03-09 18:08:20,768 : INFO : using serial LDA version on this node
2022-03-09 18:08:20,770 : INFO : running online (multi-pass) LDA training, 10 topics, 20 passes over the supplied corpus of 2695 documents, updating model once every 2000 documents, evaluating perplexity every 2695 documents, iterating 400x with a convergence threshold of 0.001000
2022-03-09 18:08:20,770 : INFO : PROGRESS: pass 0, at document #2000/2695
2022-03-09 18:08:22,590 : INFO : optimized alpha [0.06499475, 0.07717129, 0.0680591, 0.06421463, 0.07580097, 0.07692016, 0.06895801, 0.08344096, 0.07965693, 0.07317674]
2022-03-09 18:08:22,591 : INFO : merging changes from 2000 documents into a model of 2695 documents
2022-03-09 18:08:22,592 : INFO : topic #3 (0.064): 0.023*"から" + 0.022*"と" + 0.019*"際" + 0.015*"へ" + 0.014*"
" + 0.013*"足" + 0.013*"なる" + 0.013*"ある" + 0.011*"後" + 0.011*"滑る"
2022-03-09

Average topic coherence: -2.3197.
